<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/marcoteran/deeplearning/blob/master/notebooks/2.2_machinelearning_decisiontreeandrandomforests.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Abrir en Colab" title="Abrir y ejecutar en Google Colaboratory"/></a>
  </td>
  <td>
    <a target="_blank" href="https://kaggle.com/kernels/welcome?src=https://github.com/marcoteran/deeplearning/blob/master/notebooks/2.2_machinelearning_decisiontreeandrandomforests.ipynb"><img src="https://kaggle.com/static/images/open-in-kaggle.svg" alt="Abrir en Kaggle" title="Abrir y ejecutar en Kaggle"/></a>
  </td>
</table>

### Ejemplo de código
# Sesión 04: Árboles de decisión, Random Forests y exploración aleatorizada
## Deep Learning y series de tiempo

**Name:** Marco Teran **E-mail:** marco.tulio.teran@gmail.com,
[Website](http://marcoteran.github.io/),
[Github](https://github.com/marcoteran),
[LinkedIn](https://www.linkedin.com/in/marcoteran/).
___

Definimos primero unas librerías y funciones que vamos a usar a durante la sesión:

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pylab as pl
import pandas as pd
from time import time

from sklearn import datasets
from sklearn.datasets import make_circles, make_moons

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

# Para hacer gráficas
from matplotlib.colors import Normalize
import seaborn as sns

In [ ]:
# Función para visualizar un conjunto de datos en 2D
def plot_data(X, y):
    y_unique = np.unique(y)
    colors = pl.cm.rainbow(np.linspace(0.0, 1.0, y_unique.size))
    for this_y, color in zip(y_unique, colors):
        this_X = X[y == this_y]
        pl.scatter(this_X[:, 0], this_X[:, 1],  c=color,
                    alpha=0.5, edgecolor='k',
                    label="Class %s" % this_y)
    pl.legend(loc="best")
    pl.title("Data")
    
# Función para visualizar de la superficie de decisión de un clasificador
def plot_decision_region(X, pred_fun):
    min_x = np.min(X[:, 0])
    max_x = np.max(X[:, 0])
    min_y = np.min(X[:, 1])
    max_y = np.max(X[:, 1])
    min_x = min_x - (max_x - min_x) * 0.05
    max_x = max_x + (max_x - min_x) * 0.05
    min_y = min_y - (max_y - min_y) * 0.05
    max_y = max_y + (max_y - min_y) * 0.05
    x_vals = np.linspace(min_x, max_x, 100)
    y_vals = np.linspace(min_y, max_y, 100)
    XX, YY = np.meshgrid(x_vals, y_vals)
    grid_r, grid_c = XX.shape
    ZZ = np.zeros((grid_r, grid_c))
    for i in range(grid_r):
        for j in range(grid_c):
            ZZ[i, j] = pred_fun(XX[i, j], YY[i, j])
    pl.contourf(XX, YY, ZZ, 100, cmap = pl.cm.coolwarm, vmin= -1, vmax=2)
    pl.colorbar()
    pl.xlabel("x")
    pl.ylabel("y")
    
class MidpointNormalize(Normalize):

    def __init__(self, vmin=None, vmax=None, midpoint=None, clip=False):
        self.midpoint = midpoint
        Normalize.__init__(self, vmin, vmax, clip)

    def __call__(self, value, clip=None):
        x, y = [self.vmin, self.midpoint, self.vmax], [0, 0.5, 1]
        return np.ma.masked_array(np.interp(value, x, y))
    
def gen_pred_fun(clf):
    def pred_fun(x1, x2):
        x = np.array([[x1, x2]])
        return clf.predict(x)[0]
    return pred_fun

def plot_labels(n_folds, n_classes, list_labels):
    ind = np.arange(n_folds)
    width = 0.15
    
    countings = []
    for labels in list_labels:
        labels = np.array(labels)
        countings.append([np.count_nonzero(labels == x) for x in range(n_classes)])
    
    class_bars = []
    for cls in range(n_classes):
        class_bars.append([l[cls] for l in countings])
    
    fig, ax = pl.subplots()
    i = 0
    for class_bar in class_bars:
        ax.bar(ind + width*i, class_bar, width, label='Clase '+str(i))
        i += 1
        
    ax.set_xticks(ind + 2*width / 3)
    ax.set_xticklabels(['Pliegue {}'.format(k) for k in range(n_folds)])
    pl.legend(loc="best")
    pl.title("Etiquetas")
    
# Función para visualizar un conjunto de datos en 2D
def plot_data(X, y):
    y_unique = np.unique(y)
    colors = pl.cm.rainbow(np.linspace(0.0, 1.0, y_unique.size))
    for this_y, color in zip(y_unique, colors):
        this_X = X[y == this_y]
        pl.scatter(this_X[:, 0], this_X[:, 1],  c=color.reshape(1,-1),
                    alpha=0.5, edgecolor='k',
                    label="Class %s" % this_y)
    pl.legend(loc="best")
    pl.title("Data")
    
# Función para visualizar de la superficie de decisión de un clasificador
def plot_decision_region(X, pred_fun):
    min_x = np.min(X[:, 0])
    max_x = np.max(X[:, 0])
    min_y = np.min(X[:, 1])
    max_y = np.max(X[:, 1])
    min_x = min_x - (max_x - min_x) * 0.05
    max_x = max_x + (max_x - min_x) * 0.05
    min_y = min_y - (max_y - min_y) * 0.05
    max_y = max_y + (max_y - min_y) * 0.05
    x_vals = np.linspace(min_x, max_x, 100)
    y_vals = np.linspace(min_y, max_y, 100)
    XX, YY = np.meshgrid(x_vals, y_vals)
    grid_r, grid_c = XX.shape
    ZZ = np.zeros((grid_r, grid_c))
    for i in range(grid_r):
        for j in range(grid_c):
            ZZ[i, j] = pred_fun(XX[i, j], YY[i, j])
    pl.contourf(XX, YY, ZZ, 100, cmap = pl.cm.coolwarm, vmin= -1, vmax=2)
    pl.colorbar()
    pl.xlabel("x")
    pl.ylabel("y")
    
class MidpointNormalize(Normalize):

    def __init__(self, vmin=None, vmax=None, midpoint=None, clip=False):
        self.midpoint = midpoint
        Normalize.__init__(self, vmin, vmax, clip)

    def __call__(self, value, clip=None):
        x, y = [self.vmin, self.midpoint, self.vmax], [0, 0.5, 1]
        return np.ma.masked_array(np.interp(value, x, y))
    
def gen_pred_fun(clf):
    def pred_fun(x1, x2):
        x = np.array([[x1, x2]])
        return clf.predict(x)[0]
    return pred_fun

def plot_labels(n_folds, n_classes, list_labels):
    ind = np.arange(n_folds)
    width = 0.15
    
    countings = []
    for labels in list_labels:
        labels = np.array(labels)
        countings.append([np.count_nonzero(labels == x) for x in range(n_classes)])
    
    class_bars = []
    for cls in range(n_classes):
        class_bars.append([l[cls] for l in countings])
    
    fig, ax = pl.subplots()
    i = 0
    for class_bar in class_bars:
        ax.bar(ind + width*i, class_bar, width, label='Clase '+str(i))
        i += 1
        
    ax.set_xticks(ind + 2*width / 3)
    ax.set_xticklabels(['Pliegue {}'.format(k) for k in range(n_folds)])
    pl.legend(loc="best")
    pl.title("Etiquetas")

# Árboles de Decisión

A continuación, presentamos otro algoritmo de clasificación no lineal basado en árboles de decisión. Los árboles de decisión son muy intuitivos, puesto que codifican una serie de elecciones "**si esto**" o "**sino entonces**", de forma muy similar a como una persona tomaría una decisión. La gran ventaja de esta técnica es que estas elecciones se pueden aprender de forma automática desde los datos.

## Ejemplo

Considere el siguiente árbol de decisión. Este árbol de decisión describe una serie de elecciones que buscan determinar si espero (**V**) o no (**F**) por una mesa en un restaurante.

<img src="https://github.com/marcoteran/deeplearning/raw/master/notebooks/figures/decisiontree2.png" width="70%">

Con base al anterior árbol de decisión, puedo tomar la decisión de si espero o no, usando unas reglas de clasificación sencillas, por ejemplo:

* **Si** Clientes = "Lleno" **Y** EsperaEstimada = "10-30" **Y** Hambre = "No" **Entonces** Esperar="SI"
* **Si** Clientes = "Algunos" **Entonces** Esperar="SI"
* **Si** Clientes = "Lleno" **Y** EsperaEstimada = ">60" **Entonces** Esperar="NO"

## Beneficios

* Los datos de entrada requieren muy poco preprocesamiento. Los árboles de decisión pueden trabajar con variables de diferente tipo (continuas y variables) y son invariantes al escalamiento de las características. 
* Los modelos son fáciles de interpretar, los árboles pueden ser visualizados.
* El costo computacional del uso del árbol para predecir la categoría de un ejemplo es mínimo comparado con otras técnicas (Tiempo logaritmico).

## Contras

* Puede ser tan complejo, que se memoriza el conjunto de datos, por lo tanto no generaliza tan bien (**Sobreajuste**).
* Son muy sensibles al imbalance de clases (**Sesgo**).

## ¿Cómo se construye el árbol? - Algoritmo básico
* El árbol es construido de arriba hacia abajo recursivamente de forma divide y vencerás.
* Al comienzo, todos los ejemplos de entrenamiento están en la raíz.
* Los atributos son categóricos (en caso de atributos continuos, se discretizan por adelantado)
* Los ejemplos son repartidos recursivamente de acuerdo con el atributo seleccionado. 
* Los atributos de prueba son seleccionados en base a una heurística o medida estadística (ej. **ganancia de información**)
* Se detiene hasta que solo hayan ejemplos de una clase en cada nodo hoja o se haya alcanza la profundidad máxima.

## ¿Cómo seleccionar un atributo? ¿Cómo medir si una partición es buena?

Una partición ideal es aquella que divide en un nodo las muestras de una misma clase. Observemos qué pasa si usamos la variable **Cliente** para particionar nuestro conjunto de datos.

<img src="https://github.com/marcoteran/deeplearning/raw/master/notebooks/figures/split_clients.png" width="50%">


Ahora, observemos qué pasa cuando particionamos el conjunto de datos usando la variable **Tipo de restaurante**.

<img src="https://github.com/marcoteran/deeplearning/raw/master/notebooks/figures/split_type.png" width="50%">

**¿Cuál variable es mejor?**

## Implementación en Scikit-Learn

Vamos a construir un árbol que pueda clasificar entre tres tipos diferentes de flores de Iris: **Setosa, Versicolor y Virginica**. El conjunto de datos que utilizaremos es un conjunto de datos clásico utilizado para aprender los fundamentos del Machine learning.

Cargamos el conjunto de datos usando IRIS.

In [ ]:
# Importación de los datos
iris = datasets.load_iris()

Sólo utilizaremos las caracteristicas **0** y **2**, columnas corresponden a la **longitud del sépalo** y a la **longitud del pétalo** de las flores de iris

In [ ]:
X=iris.data[:,[0,2]]
y=iris.target

La implementación de Scikit-Learn se consigue con la clase `DecisionTreeClassifier`.

In [ ]:
from sklearn.tree import DecisionTreeClassifier

Ahora es el momento de construir un árbol de decisión. Scikit-learn proporciona una clase `sklearn.tree.DecisionTreeClassifier`. Vamos a considerar algunos de los siguientes parámetros:

Vamos a clasificar los tipos de flores utilizando `scikit-learn`.
`DecisionTreeClassifier` soporta varios parámetros como lo son:
* `max_depth`: Profundidad máxima del árbol. Cuanto más profundo sea el árbol, más complejas serán las reglas de decisión y más ajustado será el modelo.
* `criterion`: Medida para determinar la calidad del particionamiento generado por un atributo. Soporta coeficiente GINI y entropía. Mide la calidad de una división. Los criterios admitidos son Gini y entropía para la ganancia de información.
* `splitter`: estrategia para elegir una división en cada nodo. Admite las estrategias "mejor" y "aleatoria".
* `min_samples_split`: Controla el número mínimo de muestras que debe haber en un nodo luego de una partición. Número mínimo de muestras necesarias para dividir un nodo interno.
* `min_samples_leaf`: Controla el número mínimo de muestras que debe haber en un nodo hoja. El número mínimo de muestras necesarias para estar en un nodo hoja.
* `random_state`: controla la aleatoriedad de la división.
* `max_leaf_nodes`: Hacer crecer un árbol con max_leaf_nodes de la mejor manera posible. Los mejores nodos se definen como la reducción relativa de impurezas. Si es None, entonces un número ilimitado de nodos hoja.

**Los pasos que damos son:**

1. Importar la clase DecisionTreeClassifier.
2. Instanciar el modelo clasificador.
3. Ajustar el modelo.
4. Predecir.
5. Evaluar el rendimiento del modelo.

In [ ]:
# Definición del clasificador
classifier = DecisionTreeClassifier(max_depth=3)

Tenga en cuenta que el 'clasificador' podría tener cualquier nombre, pero debería ser intuitivo.

In [ ]:
# Parámetros del clasificador
classifier.get_params()

Vamos a entrenar con el conjunto de dattos usando el método `.fit`.

In [ ]:
# Entrenamiento
classifier.fit(X,y)

Visualizamos la superficie de decisión del clasificador

In [ ]:
pl.figure(figsize = (10, 6))    
plot_decision_region(X, gen_pred_fun(classifier))
plot_data(X, y)

Todo lo que tenemos que hacer ahora es mostrar la precisión de nuestro árbol de decisión. Para ello utilizaremos el método `.score` de `scikit-learn`

In [ ]:
# Error de clasificación
print(classifier.score(X,y))
print(1-classifier.score(X,y))

Este árbol de decición es bastante básico y no está afinado de ninguna manera.

## Visualización

Una buena  caracteristica de `scikit-learn` es que después del entrenamiento nos permite exportar el arbol de decicion como un archivo `.dot`, que podemos visualizar eon el programa `graphviz`.
Este programa está disponible de forma gratuida en el [enlace](http://www.graphviz.org).

Es posible utilizar la librería de Python. En Ubuntu, se recomienda instalarlo usando ambas líneas:
* `conda install python-graphviz` o `conda install graphviz` o `pip install graphviz`
* `sudo apt-get install graphviz`

In [ ]:
#!pip install graphviz
#!pip install pydotplus

In [ ]:
import graphviz
from sklearn.tree import export_graphviz
from six import StringIO
from IPython.display import Image
import pydotplus
from pydotplus import graph_from_dot_data

A continuación, vamos a usar el **conjunto de datos IRIS completo** (Usando las cuatro características) y entrenamos un árbol de decisión.

In [ ]:
# Cargamos todo el conjunto de datos
X = iris.data
y = iris.target

In [ ]:
# Definición del clasificador y realizamos el entrenamiento
classifier = DecisionTreeClassifier()
classifier = classifier.fit(X,y)

Usamos `graphviz` para visualizar el árbol generado. `graphviz` soporta como parámetros los nombres de las clases y de las características

In [ ]:
dot_data = export_graphviz(classifier, out_file=None,
                           feature_names=iris.feature_names,
                           class_names=iris.target_names,
                           filled=True, rounded=True,
                           special_characters=True)
                        
graph = graphviz.Source(dot_data,"PNG")

El árbol de decisión aprendido puede ser visualizado usando `graphviz`.

In [ ]:
# Visualisamos el árbol de decición
graph

In [ ]:
# Guardarla en un archivo de imagen
graph.render("arboldedecision")

## Importancia de las variables

Una de las ventajas de usar árboles de decisión, es que nos permite determinar la importancia de cada características, con base al índice de impureza usado. Scikit-Learn nos permite acceder a la importancia de cada característica llamando `.feature_importances_`. Esta importancia cuantifica qué tanto aporta cada característica a mejorar el desempeño del árbol.

In [ ]:
classifier.feature_importances_

In [ ]:
# Nombres de las características
iris.feature_names

¿Cuál es la característica más importante?

## Evaluación de la complejidad usando `DecisionTreeClassifier`

Para evaluar la complejidad, vamos a estimar el número subóptimo de profundidad del árbol.

A continuación, dividiremos los datos en dos conjuntos: uno de entrenamiento y otro de prueba.
* El conjunto de entrenamiento se utilizará para construir el árbol de decisión, mientras que el conjunto de prueba se utilizará para comprobar la precisión del árbol de decisión.

In [ ]:
# Dividimos los datos en testeo y evaluación
X_train, X_test, y_train, y_test = train_test_split(X,y,
                                                    test_size=0.3,
                                                    random_state=1234,
                                                    stratify=y)

Vamos a explorar los siguientes valores de profundidad máxima:
* $[1, 2, 3, \dots, 20]$

In [ ]:
train_error=[]
generalization_error=[]

max_depth_values=list(range(1,21,1))

for depth in max_depth_values:
    decision_tree=DecisionTreeClassifier(max_depth=depth)
    decision_tree.fit(X_train, y_train)
    train_error.append(1-decision_tree.score(X_train, y_train))
    generalization_error.append(1-decision_tree.score(X_test, y_test))

Visualizamos la curva de error de entrenamiento contra error de generalización

In [ ]:
pl.figure(figsize = (10, 6))

pl.plot(max_depth_values, train_error, label="Entrenamiento")
pl.plot(max_depth_values, generalization_error, label="Generalización")
pl.xticks(max_depth_values)
pl.xlabel("Profundidad máxima")
pl.ylabel("Error")
pl.arrow(2.2, 0.07, 0, -0.01, head_width=0.2, head_length=0.01, fc='k', ec='k')
pl.text(2, 0.08, 'Punto de balance')
pl.legend();

## Esamble de clasificadores

Revisemos el siguiente problema.

Considere el siguiente conjunto de datos

In [ ]:
# Cargue las funciones datos make_circles n_samples=1000, factor=.4 (separación entre circulos), noise=.15
np.random.seed(0)
X, y = make_circles(n_samples=1000, factor=.4, noise=.15)

In [ ]:
# Dividamos el conjunto de datos 60/40
X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    test_size=0.4,
                                                    random_state=2)

In [ ]:
pl.figure(figsize = (10, 6))
plot_data(X, y)

Vamos a entrenar un modelo de árbol de decisión para resolver este problema de clasificación.

Entrenamos un árbol de decisión de profundidad máxima $5$

In [ ]:
dt = DecisionTreeClassifier(max_depth=5)

dt.fit(X_train, y_train)

In [ ]:
print('Error en entrenamiento: {}'.format(1-dt.score(X_train, y_train)))
print('Error en prueba: {}'.format(1-dt.score(X_test, y_test)))

pl.figure(figsize = (10, 6))
plot_decision_region(X_test, gen_pred_fun(dt))
plot_data(X_test, y_test)

El modelo presenta un error de generalización del $6\%$, además sus predicciones no se ajustan a la naturaleza de los datos

Problemas:
* En algunos casos, las fronteras de decisión generadas por el árbol de decisión (paralelas a los ejes) no son lo suficientemente flexibles para capturar la no linealidad del conjunto de datos.
* Por otro lado, los árboles de clasificación pueden crear reglas decisión muy espcíficas que se ajustan demasiado a los datos de entrenamiento.
* Tambien son sensibles a pequeñas variaciones de los datos que pueden resultar en árboles totalmente distintos. Esto es un problema cuando se hace validación cruzada de k-pliegues.

**¿Cómo solucionar este problema?**

Vamos a utilizar una estrategia conocida como *clasificación por comité* o *ensamble de clasificadores* (ensemble). Un ensamble de clasificadores combina diferentes algoritmos de aprendizaje para obtener un mejor desempeño predictivo.

### Bagging

Bagging es una técnica de ensamblaje de modelos que se utiliza en Machine Learning y consiste en:
- Entrenar múltiples modelos de aprendizaje automático en diferentes subconjuntos de datos de entrenamiento seleccionados al azar.
- Combinar las predicciones de todos los modelos en una única predicción mediante una votación ponderada o promedio de las predicciones individuales.
- Esta técnica ayuda a reducir el sobreajuste y mejorar la precisión del modelo.
- Bagging se puede utilizar con cualquier modelo de aprendizaje automático, aunque se utiliza con mayor frecuencia con árboles de decisión.
- Es una técnica útil para mejorar la estabilidad del modelo y puede ser especialmente útil en problemas de clasificación donde las clases están desequilibradas.

A continuación, presentamos una forma de como construír un ensamble de árboles de clasificación. Se entrenan diferentes árboles en diferentes subconjuntos de los datos de entrenamiento. Posteriormente cuando se va hacer una predicción para un nuevo dato, se obtienen las predicciones de todos los árboles y se regresa aquella clase de mayor frecuencia predicha (votación).

<img src="https://github.com/marcoteran/deeplearning/raw/master/notebooks/figures/ensemble2.svg" width="50%">

A la técnica de la figura se le conoce como *bagging*.

En el ejemplo siguiente vamos a utilizar la función `BaggingClassifier`, la cual entrena un modelo usando esta estrategia.

Los parámetros recibidos por la función son los siguientes:
* `base_estimator`: Consiste en el estimador base que se va a entrenar sobre los diferentes subconjunto del conjunto de datos de entrenamiento.
* `n_estimators`: Es el número de estimadores base que se van a usar para la estrategia de *bagging*.

In [ ]:
from sklearn.ensemble import BaggingClassifier

bc = BaggingClassifier(base_estimator=DecisionTreeClassifier(max_depth=5), n_estimators=20)
bc.fit(X_train, y_train)

In [ ]:
print('Error en entrenamiento: {}'.format(1-bc.score(X_train, y_train)))
print('Error en prueba: {}'.format(1-bc.score(X_test, y_test)))

pl.figure(figsize = (10, 6))
plot_decision_region(X_test, gen_pred_fun(bc))
plot_data(X_test, y_test)

# Random Forests

Es un método de ensamble que combina múltiples árboles de decisión.
* Cada árbol se entrena con un subconjunto aleatorio de características y datos.
* La predicción final se realiza promediando las predicciones de todos los árboles individuales.
* Random Forest es efectivo para prevenir el sobreajuste y es útil para clasificación y regresión.

Existen varias alternativas para combinar y promediar árboles de decisión, una de las más usadas es **Random Forest**.
Vamos a construir un simple bosque aleatorio en Python.

<img src="https://github.com/marcoteran/deeplearning/blob/master/notebooks/figures/randomforest.png?raw=1" width="60%">

Nuestro *forest* será muy similar al *árbol de decisión* que hicimos anteriormente.
Esto significa que vamos a construir un *random forest* que pueda clasificar entre tres tipos diferentes de flores "Iris": *Setosa, Versicolor y Virginica*.

`Scikit-Learn` provee una implementación a través de `sklearn.ensemble.RandomForestClassifier`. Más adelante se discuten los parámetros más importantes de esta implementación.

### Algoritmo básico Random Forest

A diferencia del árbol común de decisión, los árboles de Random Forest se entrenan de una forma diferente. A continuación, presentamos el algoritmo básico de entrenamiento:

1. Para cada árbol se realiza el siguiente procedimiento:
    * Se escoge una muestra con reemplazo de tamaño $n$ del conjunto de entrenamiento.
    * Se seleccionan $m$ variables al azar de las variables disponibles
    * Se entrena un árbol sobre la muestra usando las $m$ variables repitiendo los siguientes pasos:
        * Se escoge la variable (y umbral) que genera la mejor partición
        * Se divide los datos en dos subconjuntos de acuerdo a la variable y el umbral
        * Si no se satisface un criterio de parada se aplica este procedimiento recursivamente sobre los subconjutos
2. Una vez cada árbol ha sido entrenado, se genera el ensamble de árboles.

### Implementación en Scikit-Learn

La implementación en Scikit-Learn nos permite controlar los siguientes parámetros:
* `n_estimators`: Número de árboles a entrenar
* `max_features`: Número de variables $m$ al azar que se tienen en cuenta para la construcción de cada árbol.

Un número grande de árboles resulta en un buen desempeño a costas del costo computacional. Ambos parámetros deben ser explorados usando validación cruzada.

In [ ]:
from sklearn.ensemble import RandomForestClassifier

Utilizaremos una semilla aleatoria `np.random.seed` en cero.
La semilla aleatoria es un punto de partida para generar números aleatorios y replicarlos en otras ocasiones los mismos resultados.

In [ ]:
np.random.seed(0)

Usaremos scikit-learn para crear un clasificador *random forest* que llamaremos `rf`, pero podría llamarse de cualquier manera.

In [ ]:
rf = RandomForestClassifier(n_estimators=20, max_depth=5)

Entrenaremos el clasificador utilizando las características que hemos definido antes sobre los datos y las etiquetas

In [ ]:
rf.fit(X_train, y_train)

Dibujamos la superficie de decisión junto a los datos de testeo

In [ ]:
pl.figure(figsize = (10, 6))
plot_decision_region(X_test, gen_pred_fun(rf))
plot_data(X_test, y_test)

In [ ]:
print('Error en entrenamiento: {}'.format(1-rf.score(X_train, y_train)))
print('Error en prueba: {}'.format(1-rf.score(X_test, y_test)))

Mientras baja el error de generalización, observamos tambien que se adapta mejor a la naturaleza de los datos

### Intuición detrás de RandomForests

Random Forest es una técnica de **ensamble** que combina diferentes árboles de decisión. Estos árboles se entrenan en diferentes muestras del conjunto de datos, estos árboles pueden sobreajustarse, sin embargo la combinación de sus predicciones resulta en un clasificador con menor sobreajuste.

Para entender la idea de como se construye este algoritmo, suponga que entrenamos los siguientes árboles para determinar si una persona está enferma o sana:

<img src="https://github.com/marcoteran/deeplearning/raw/master/notebooks/figures/RF.png" width="70%">

**¿Qué sucede si todos los árboles de clasificación son iguales?**

**¿Cómo logro que cada árbol de clasificación sea diferente?**

## Ajuste de hiperparámetros del esamble

### Cargamos el dataset

Utilizaremos el conjunto de datos de clasificación de vino.
* El conjunto de datos  que contiene los resultados de un análisis químico de vinos cultivados en un área específica de Italia.
* Tres tipos de vino están representados en las 178 muestras, con los resultados de 13 análisis químicos registrados para cada muestra.

A continuación, usaremos el conjunto de datos `wine`.

In [ ]:
#!mkdir data
#!wget -O data/wine.data.txt https://github.com/marcoteran/deeplearning/raw/master/notebooks/data/wine.data.txt

In [ ]:
df = pd.read_csv('data/wine.data.txt')

In [ ]:
# Mostremos los primeros 5 datos
df.head()

In [ ]:
# comprobar la frecuencia de los valores únicos en la columna de calidad `quality`
df['Class'].value_counts()

En primer lugar, dividamos el conjunto de datos en variables independientes y dependientes. La característica `Class` será la variable objetivo. Generamos la matriz de características y el arreglo de etiquetas

In [ ]:
X = df.drop(labels='Class', axis=1).values
y = df['Class'].values

### Creación de los conjuntos de entrenamiento y prueba
Divida el conjunto de datos en un conjunto de entrenamiento y otro de prueba.

In [ ]:
# Dividimos los datos estratificados al 70/30
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1, stratify=y)

Usaremos scikit-learn para crear un clasificador *random forest* que llamaremos `rf`, pero podría llamarse de cualquier manera.

In [ ]:
rf = RandomForestClassifier(n_estimators=20, max_depth=5)

Entrenaremos el clasificador utilizando las características que hemos definido antes sobre los datos y las etiquetas

In [ ]:
rf.fit(X_train, y_train)

In [ ]:
print('Error en entrenamiento: {}'.format(1-rf.score(X_train, y_train)))
print('Error en prueba: {}'.format(1-rf.score(X_test, y_test)))

In [ ]:
# Hacer las prediccciones
y_pred = rf.predict(X_test) # Test the prediction accurracy of the model
result = pd.DataFrame({'Actual' : y_test, 'Predicted' : y_pred})
pd.concat([result.head(), result.tail()]) # display df of acutal and predicted(head and tail)

In [ ]:
from sklearn.metrics import accuracy_score
print('Testing Set Evaluation Accuracy: ',accuracy_score(y_test,y_pred))

In [ ]:
from sklearn.metrics import confusion_matrix

cf_matrix = confusion_matrix(y_test, y_pred) # generate confusion matrix
sns.heatmap(pd.DataFrame(cf_matrix), annot=True, cmap='RdYlBu', linewidth=.5, fmt='g', cbar=False)
plt.title('Confusion matrix', y=1.1)
plt.ylabel('Actual label')
plt.xlabel('Predicted label')
cf_matrix

Informe de clasificación:

In [ ]:
from sklearn.metrics import classification_report

target_names = ['1', '2', '3']

pd.DataFrame(classification_report(y_test, y_pred,target_names=target_names, output_dict=True))

Tenemos la oportunidad de mejorar el rendimiento del modelo mediante el ajuste de hiperparámetros.
* En el ajuste de hiperparámetros, especificamos los posibles parámetros óptimos para optimizar el rendimiento del modelo
Dado que es imposible conocer manualmente los parámetros óptimos para nuestro modelo, vamos a automatizarlo utilizando la clase `sklearn.model_selection.GridSearchCV`.

Veamos cómo podemos realizar esto en un Clasificador de Árbol de Decisión.
Utilizando la clase `GridSearchCV` de sklearn y le pasaremos valores predefinidos para los hiperparámetros.

Definimos la malla de parámetros:
* `max_features_params = [np.round(10**-1 * i, decimals=1) for i in range(1, 11, 1)]`
* `param_grid = {'n_estimators': [2**i for i in range(2, 12, 1)], 'max_features': max_features_params}`

In [ ]:
max_features_params = [np.round(10**-1 * i, decimals=1) for i in range(1, 11, 1)]
param_grid = {'n_estimators': [2**i for i in range(2, 12, 1)], 'max_features': max_features_params}

In [ ]:
print('Número de árboles: {}'.format(param_grid['n_estimators']))

In [ ]:
print('Porcentaje de características a usar: {}'.format(param_grid['max_features']))

Corremos el modelo `GridSearchCV` usando la retícula de parámetros

In [ ]:
start = time()
clf = GridSearchCV(RandomForestClassifier(), param_grid=param_grid, verbose=1, n_jobs=-1, cv=5)
clf.fit(X_train, y_train)
print("GridSearchCV tomó {} segundos usando {} configuraciones".format(time() - start,
                                                                         len(clf.cv_results_['params'])))

Usando `cv_results_` extraemos el desempeño promedio sobre cada configuración de parámetros

In [ ]:
scores = clf.cv_results_['mean_test_score'].reshape(len(param_grid['max_features']),
                                                    len(param_grid['n_estimators']))

In [ ]:
scores

Visualizamos la mejor combinación de parámetros:

In [ ]:
pl.figure(figsize=(10, 6))
pl.subplots_adjust(left=.2, right=0.95, bottom=0.15, top=0.95)
pl.imshow(scores, interpolation='nearest', cmap=plt.cm.hot,
           norm=MidpointNormalize(vmin=0.89, midpoint=0.97, vmax=1.))
pl.xlabel('n_estimators')
pl.ylabel('max_features')
pl.colorbar()
pl.xticks(np.arange(len(param_grid['n_estimators'])), param_grid['n_estimators'], rotation=45)
pl.yticks(np.arange(len(param_grid['max_features'])), param_grid['max_features'])
pl.title('Accuracy en validación')
pl.show()

La mejor combinación de parámetros está dada por:

In [ ]:
clf.best_params_

In [ ]:
clf.best_score_

Finalmente, reportamos en el conjunto de prueba:

In [ ]:
clf.score(X_test, y_test)

## Importancia de características

Una ventaja muy significativa de usar Random Forests consiste en la posibilidad de obtener la importancia de las características del conjunto de datos. Esta importancia nos indica qué tanto una característica contribuye al desempeño en los nodos de los diferentes árboles.

A continuación, seguimos usando el conjunto de datos `wine` y obtenemos la importancia de las características del mejor modelo usando validación cruzada. Entrenemos de nuevo el modelo:

In [ ]:
clf = RandomForestClassifier(n_estimators=64, max_features=.1)

clf.fit(X_train, y_train);

Extraemos la importancia de las características

In [ ]:
clf.feature_importances_

A continuación ordenamos las características por su importancia

In [ ]:
importances = clf.feature_importances_
indices = np.argsort(importances)[::-1]

print("Importancia de características:")

for f in range(X_train.shape[1]):
    print("Característica %s (%f)" % (df.columns[int(1+indices[f])], importances[indices[f]]))

In [ ]:
plt.figure()
plt.title("Importancia de las características")
plt.bar(range(X_train.shape[1]), importances[indices],
       color="r", align="center")
xticks_labels = [df.columns[1+i] for i in indices]
plt.xticks(range(X.shape[1]), xticks_labels, rotation=45)
plt.xlim([-1, X.shape[1]])
plt.show()

# Optimización aleatoria de parámetros

A pesar de las ventajas que ofrece `GridSearchCV` sobre la exploración sistemática de los hiperparámetros de un modelo, puede gastar un tiempo considerable en esta exploración. Scikit-Learn permite hacer también una exploración aleatoria de los parámetros, la cual se ha demostrado empiricamente que es más eficiente que optimizar los parámetros usando una malla con `GridSearchCV` ([Referencia](http://www.jmlr.org/papers/volume13/bergstra12a/bergstra12a.pdf)). `RandomizedSearchCV` implementa una búsqueda aleatoria sobre los parámetros. El rango de exploración de los parámetros se puede especificar de la siguiente manera:

* Usando una lista:
    * `"criterion": ["gini", "entropy"]`
* Usando una distribución discreta uniforme:
    * `"n_estimators": randint(4, 2048)`
    * `randint` proviene de `scipy.stats`. Este genera una distribución discreta entre 4 y 2048. 
* Usando una distribución uniforme:
    * `"max_features": uniform()`
    * `uniform` proviene de `scipy.stats`. Este genera una distribución uniforme entre 0 y 1.
    
`RandomizedSearchCV` puede recibir tanto listas de elementos como distribuciones de probabilidad, las cuales deben ser especificadas usando `scipy.stats`. A continuación, creamos nuestro estimador y definimos la distribución de parámetros:

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform 
from scipy.stats import randint

clf = RandomForestClassifier()

param_dist = {"n_estimators": randint(4, 800),
              "max_features": uniform()}

In [ ]:
b=randint(4, 800)

In [ ]:
b

`n_iter_search` nos define el número de configuraciones que vamos a extraer de la distribución de parámetros.

In [ ]:
n_iter_search = 20
random_search = RandomizedSearchCV(clf, param_distributions=param_dist,
                                   n_iter=n_iter_search, cv=5, 
                                   n_jobs=-1, verbose=2)

`RandomizedSearchCV` también soporta la ejecución en paralelo usando `n_jobs=-1`. También podemos especificar el número de pliegues a usar usando `cv=5`

In [ ]:
start = time()
random_search.fit(X_train, y_train)
print("RandomizedSearchCV tomó {} segundos usando {} configuraciones".format(time() - start,
                                                                               n_iter_search))

Para validar esta información, `GridSearchCV` nos ofrece una serie de métodos que nos permite consultar:
* La lista de resultados por elemento en la malla de parámetros (`cv_results_`)
* La configuración con el mejor desempeño (`best_params_`)
* El accuracy promediado sobre todos los pliegues de la mejor configuración (`best_score_`)

Para encontrar las mejores configuraciones, ordenamos la tabla de resultados de la siguiente manera:

Extraemos los resultados con mejor desempeño promedio:

In [ ]:
cv_results = pd.DataFrame(random_search.cv_results_)
cv_results = cv_results[['param_n_estimators','param_max_features','mean_test_score']]
cv_results.sort_values(by='mean_test_score',ascending=False).head()

Verificamos la mejor configuración y su puntaje sobre todas las particiones de validación:

In [ ]:
random_search.best_params_

In [ ]:
random_search.best_score_

Reportamos el error de generalización:

In [ ]:
random_search.score(X_test, y_test)

___
¡Todo bien! ¡Es todo por hoy! 😀

___

# Taller

Este taller se va a enfocar en trabajar con el conjunto de datos del censo de 1993 de USA. El conjunto de datos reune una serie de características socioeconómicas tanto categóricas como numéricas. El objetivo es construir un clasificador que determine si la persona tiene una ganancia alta al año (más de 50K USD) o baja (menos de 50K USD al año). A continuación encuentra una descripción de las características:

Etiqueta:
* income: >50K, <=50K.
Características:
* age: Variable continua.
* workclass: Private, Self-emp-not-inc, Self-emp-inc, Federal-gov, Local-gov, State-gov, Without-pay, Never-worked.
* fnlwgt: Variable continua.
* education: Bachelors, Some-college, 11th, HS-grad, Prof-school, Assoc-acdm, Assoc-voc, 9th, 7th-8th, 12th, Masters, 1st-4th, 10th, Doctorate, 5th-6th, Preschool.
* education-num: Variable continua.
* marital-status: Married-civ-spouse, Divorced, Never-married, Separated, Widowed, Married-spouse-absent, Married-AF-spouse.
* occupation: Tech-support, Craft-repair, Other-service, Sales, Exec-managerial, Prof-specialty, Handlers-cleaners, Machine-op-inspct, Adm-clerical, Farming-fishing, Transport-moving, Priv-house-serv, Protective-serv, Armed-Forces.
* relationship: Wife, Own-child, Husband, Not-in-family, Other-relative, Unmarried.
* race: White, Asian-Pac-Islander, Amer-Indian-Eskimo, Other, Black.
* sex: Female, Male.
* capital-gain: Variable continua.
* capital-loss: Variable continua.
* hours-per-week: Variable continua.
* native-country: United-States, Cambodia, England, Puerto-Rico, Canada, Germany, Outlying-US(Guam-USVI-etc), India, Japan, Greece, South, China, Cuba, Iran, Honduras, Philippines, Italy, Poland, Jamaica, Vietnam, Mexico, Portugal, Ireland, France, Dominican-Republic, Laos, Ecuador, Taiwan, Haiti, Columbia, Hungary, Guatemala, Nicaragua, Scotland, Thailand, Yugoslavia, El-Salvador, Trinadad&Tobago, Peru, Hong, Holand-Netherlands.


## Carga de datos
* Cargue el conjunto de datos `adult.csv` en Pandas.
* Este conjunto de datos tiene dos problemas:
    * Contiene datos faltantes, representados por un interrogante: "?"
    * Es un conjunto de datos imbalanceado.
* Para verificar ambos hechos:
    * Verifique qué porcentaje de valores indefinidos hay para las variables `workclass`, `occupation` y `native-country`.
    * Luego, puede limpiar aquellas filas que contengan datos indefinidos. Puede usar el siguiente código:
    ```python
    df = df[df["workclass"] != "?"]
    df = df[df["occupation"] != "?"]
    df = df[df["native-country"] != "?"]
    ```
    * Verifique el DataFrame tenga un tamaño de 45222 elementos por 15 características.
    * Verifique la distribución de etiquetas en la columna `income`, ¿Cuál clase tiene mayor número de ejemplos?
* Use `.describe()` para obtener un análisis de las variables numéricas del conjunto de datos.
* Realice un conteo de cada elemento de las variables categóricas:
    * ['workclass', 'race', 'education','marital-status', 'occupation','relationship', 'gender', 'native-country', 'income'] 
* Simplifique la categoría 'marital-status' de la siguiente forma:
    * 'Divorced' -> 'not married'
    * 'Married-AF-spouse' -> 'married'
    * 'Married-civ-spouse' -> 'married'
    * 'Married-spouse-absent' -> 'married'
    * 'Never-married' -> 'not married'
    * 'Separated' -> 'not married'
    * 'Widowed' -> 'not married'
* Convierta las variables categóricas a numéricas. El enfoque aquí presentado no es el mejor, pero será una primera aproximación. Puede usar el siguiente código:
```python
for col in category_col:
    b, c = np.unique(df[col], return_inverse=True) 
    df[col] = c
```
A pesar de que no queremos inducir un orden en las categorías analizadas, es un buen inicio para probar Validación cruzada.
* Cree una matriz `X` de características usando:
    * ['age','workclass','education','educational-num','marital-status', 'occupation','relationship','race','gender','capital-gain','capital-loss','hours-per-week', 'native-country']
* Cree el arreglo `y` usando la columna `income`:
* Genere una partición de entrenamiento y prueba $80\%-20\%$ estratificada. NO MODIFIQUE EL CONJUNTO DE DATOS ORIGINAL.
* Verifique que en efecto la partición haya sido estratificada.

## RandomizedSearchCV

* Genere la siguiente distribución de parámetros:
    * `n_estimators` seguirá una distribución uniforme discreta entre 4 y 512
    * `max_features` seguirá una distribución uniforme entre 0 y 1
    * `max_depth` será un valor entre 5 o None
    * `bootstrap` será un valor entre True o False
* ¿Qué hace `max_depth` y `bootstrap`? ¿Qué significa que `max_depth` sea None? Consulte la [documentación](http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html)
* Entrene un RandomForestClassifier sobre las siguientes condiciones:
    * 20 iteraciones de la distribución de parámetros
    * Pliegues de la validación cruzada= 5
    * Use `n_jobs=-1` para hacer la búsqueda de parámetros de forma paralela
* ¿Cuales son las mejores configuraciones?
* ¿Qué desempeño tienen las mejores configuraciones?
* Reporte accuracy, precision, recall y matriz de confusión en el conjunto de prueba